In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'

import jax
import jax.numpy as jnp
from jax import random
import matplotlib.pyplot as plt

import jax.random as random
rng = random.key(0)

from phyEBM.utils import  get_keys_and_rng

In [ ]:
x1 = jnp.linspace(0., 1., 500)
x2 = x1
X = jnp.stack(jnp.meshgrid(x1,x2, indexing='ij'))
print(X.shape)
i_complex = jax.lax.complex(0., 1.)
key, rng = random.split(rng)

n, m  = 20, 20
key1, key2, rng = random.split(key, num=3)

alpha_indices = jnp.stack(jnp.meshgrid( jnp.arange(0, stop=n), jnp.arange(0, stop=m), indexing='ij'))
alpha = random.normal(key1, shape=(n,m))
basis = lambda alpha_nm, n, m, X0, X1: alpha_nm  * jnp.cos(jnp.pi*n*X0) *  jnp.cos(jnp.pi*m*X1)


f_vals = jnp.sum(jax.vmap(basis, in_axes=(0,0,0, None, None))(alpha.reshape(-1,), alpha_indices[0].reshape(-1,), alpha_indices[1].reshape(-1,), X[0], X[1]), axis=0)
print(f_vals.shape)
f_vals = jnp.real(f_vals.reshape(X.shape[1:]))
plt.contourf(X[0], X[1], f_vals, 50)
plt.colorbar()
plt.show()

In [ ]:
m_l = 2
key, rng = random.split(rng)

kappas = jnp.array([1., 2.])

level_set_func = lambda f_vals, i: jnp.where(f_vals < 0, kappas[0], kappas[1])
level_set_field = jnp.sum(jax.vmap(level_set_func, in_axes=(None, 0))(f_vals, jnp.arange(0, stop=m_l)), axis=0)

plt.contourf(X[0], X[1], level_set_field, 50)
plt.colorbar()
plt.show()

In [ ]:
n, m  = 5, 5
key1, key2, rng = random.split(key, num=3)

alpha_indices = jnp.stack(jnp.meshgrid( jnp.arange(1, stop=n+1), jnp.arange(1, stop=m+1), indexing='ij'))
alpha = random.normal(key1, shape=(n,m))
lambda_val = 0.5
beta       = 1.
rand_basis = lambda key, alpha_nm, n, m, X0, X1: ((n**2.+m**2.)*jnp.pi**2. + lambda_val**2.)**(-beta) * random.normal(key) * jnp.cos(jnp.pi*n*X0) *  jnp.cos(jnp.pi*m*X1)


keys, rng = get_keys_and_rng(rng, num=n*m)  
f_vals = jnp.sum(jax.vmap(rand_basis, in_axes=(0, 0,0,0, None, None))(keys, alpha.reshape(-1,), alpha_indices[0].reshape(-1,), alpha_indices[1].reshape(-1,), X[0], X[1]), axis=0)
print(f_vals.shape)
f_vals = jnp.real(f_vals.reshape(X.shape[1:]))
plt.contourf(X[0], X[1], f_vals, 20)
plt.colorbar()
plt.show()

m_l = 2
key, rng = random.split(rng)

kappas = jnp.array([1., 2.])

level_set_func = lambda f_vals, i: jnp.where(f_vals < 0, kappas[0], kappas[1])
level_set_field = jnp.sum(jax.vmap(level_set_func, in_axes=(None, 0))(f_vals, jnp.arange(0, stop=m_l)), axis=0)

plt.contourf(X[0], X[1], level_set_field, 50)
plt.colorbar()
plt.show()

In [ ]:
x = jnp.linspace(0., 1., 500)
from jax import random
key, rng = random.split(rng)

N = 4
lambda_val = 20.
beta = 2.
keys, rng = get_keys_and_rng(key, num=N)  
basis = lambda key, n, x: (n**2.*jnp.pi**2.+lambda_val**2.)**(-beta) * random.normal(key) * jnp.cos(jnp.pi*n*x)
f_vals = jnp.sum(jax.vmap(basis, in_axes=(0, 0, None))(keys, jnp.arange(1, stop=N+1), x), axis=0)

# basis = lambda alpha_n, n, x: alpha_n * jnp.cos(jnp.arccos(x)*n)
# f_vals = jnp.sum(jax.vmap(basis, in_axes=(0,0, None))(alpha, jnp.arange(0, stop=alpha.shape[0]), x), axis=0)

plt.plot(x, f_vals)
plt.show()

l =10
kp, km = 2., 1.
def z(a):
    # a_bar = jnp.max(jnp.abs(a))
    a_bar = jnp.sqrt(jnp.sum(a**2.*(x[1]-x[0])))
    return 0.5 * jnp.tanh(l * a/a_bar)*(kp-km) + 0.5 * (kp - km) + km
plt.plot(x, z(f_vals))
plt.show()

In [ ]:
j = jnp.arange(1, stop=20+1)
lambda_val = 1.
betas = jnp.linspace((1.), (4.), 10)
ax = plt.axes() 
for beta in betas:
    vals = (2*j**2.*jnp.pi**2. + lambda_val**2.)**(-beta)
    plt.plot(j,  vals/vals[0], '-o', label=f'{beta:.2f}')
    ax.set_xticks(j) 
plt.title(rf'$\beta$;   $\quad\lambda={lambda_val}$')
plt.ylabel(r'$\frac{((j^2+k^2)\pi^2+\lambda^2)^{-\beta}}{(2\pi^2+\lambda^2)^{-\beta}}$')
plt.xlabel(r'$j=k$')
plt.grid()
plt.legend()
plt.show()

beta = 2.
lambda_vals = jnp.exp(jnp.linspace(jnp.log(0.5), jnp.log(25.), 10))
ax = plt.axes() 
for lambda_val in lambda_vals:
    vals = (2*j**2.*jnp.pi**2. + lambda_val**2.)**(-beta)
    plt.plot(j, vals/vals[0], '-o', label=f'{lambda_val:.2f}')
    ax.set_xticks(j) 
    # ax.set_xticklabels(j**2) 
plt.title(rf'$\lambda$; $\quad\beta={beta}$')
plt.ylabel(r'$\frac{((j^2+k^2)\pi^2+\lambda^2)^{-\beta}}{(2\pi^2+\lambda^2)^{-\beta}}$')
plt.xlabel(r'$j=k$')
plt.legend()
plt.grid()
plt.savefig('TwoDSpectrumDecay_GRF.pdf')
plt.show()

#relative last value eigenvalue
lambda_vals = jnp.exp(jnp.linspace(jnp.log(0.5), jnp.log(40.), 100))
J, K = 20,20
rel_last_eing = ((J**2.+K**2.)*jnp.pi**2.+lambda_vals**2.)**(-beta) / (2*jnp.pi**2. + lambda_vals**2.)**(-beta)
plt.plot(lambda_vals, rel_last_eing)
plt.grid()
plt.title(rf'truncated mode relative stddev; $\beta={beta}$, $J=K={K}$')
plt.ylabel(r'$\frac{((J^2+K^2)\pi^2+\lambda^2)^{-\beta}}{(2\pi^2+\lambda^2)^{-\beta}}$')
plt.xlabel(r'$\lambda$')
plt.show()

In [ ]:
j = jnp.arange(1, stop=20+1)

beta = 4.
lambda_vals = jnp.exp(jnp.linspace(jnp.log(0.5), jnp.log(25.), 10))
ax = plt.axes() 
for lambda_val in lambda_vals:
    vals = (j**2.*jnp.pi**2. + lambda_val**2.)**(-beta/2)
    plt.plot(j, vals/vals[0], '-o', label=f'{lambda_val:.2f}')
    ax.set_xticks(j) 
    # ax.set_xticklabels(j**2) 
plt.title(rf'$\lambda$; $\quad\beta={beta}$')
plt.ylabel(r'$\frac{(j^2\pi^2+\lambda^2)^{-\beta}}{(\pi^2+\lambda^2)^{-\beta}}$')
plt.xlabel(r'$j$')
plt.rcParams.update({
    'axes.labelsize':   16,
    'axes.titlesize':   16,
    'xtick.labelsize' : 12,
    'ytick.labelsize' : 12,
          })
plt.legend()
plt.grid()
plt.savefig(f'OneDSpectrumDecay_GRF_beta{beta}.pdf')
plt.show()